 ╔══<i><b>Alai-DeepLearning</b></i>════════════════════════════╗
###  &nbsp;&nbsp; **✎&nbsp;&nbsp;Week 9. CNN Basis**
# Section 5. Lenet-5 구성하기

### _Objective_
1. LeNet-5은 1980년대 convolution neural network로 혁신을 가져왔습니다. <br>
2. Tensorflow의 Low-API를 이용하여, LeNet-5을 구성하도록 하겠습니다. <br> 
  
╚══════════════════════════════════════════╝

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tqdm import tqdm

## Graph Visualization

In [0]:
from IPython.display import clear_output, Image, display, HTML
import numpy as np    

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

<br><br>

# \[ 1. Tensorflow를 이용한 Lenet-5 구성하기 \]

----
----
 
> *아래는 논문에서 발췌한 Lenet-5의 구조도입니다.*<br> 

![Imgur](https://i.imgur.com/WCN9wTP.png)



| 층  | 종류 |필터 갯수 | 필터 크기 | 스트라이드 | 패딩 |
|--- |--- |----| ----|----|----|
| c1 |합성곱| 6   | (5,5) | 1  | valid |
| s2 |pooling| --- | (2,2) | 2  | valid | 
| c3 |합성곱| 16  | (10,10) | 1 | valid |
| s4 |pooling| --- | (2,2) | 2 | valid |
| c5 |합성곱 | 120  | (5,5) | 1  | valid |
| f6 |FC   | 84   | --- | ---  | --- |
| OUT | softmax | 10 |  --- | --- | --- |



<br>

## 1. Tensorflow Graph 구성하기
----

위의 모델은 1980년대 만들어진 모델로, 현대의 딥러닝 발전에 매우 커다란 공헌을 하였습니다.<br>
하지만 현대 딥러닝 모델을 구성할 때의 방식과는 약간씩 다른 부분들이 있습니다.<br>

### (1) placeholder 구성하기

LeNet에서는 Input size가 Mnist 데이터셋의 기본 크기인 (28,28)이 아닌,<br>
여기에 Zero-padding을 더해 32x32 크기로 구성하였습니다.<br>
이는 각 convolution 단계에서 조금씩 이미지가 줄어드는 것을 방지하고자 함이었습니다.<br>


In [0]:
graph = tf.Graph()

with graph.as_default():
    x = tf.placeholder(tf.float32,(None,28,28,1),name='RAW_INPUT')
    
    with tf.variable_scope('padding'):
        paddings = tf.constant([[0,0],[2,2],[2,2],[0,0]])
        x_with_pad = tf.pad(x, paddings)
    x_with_pad = tf.identity(x_with_pad,name='INPUT')

In [0]:
show_graph(graph)

tf.pad는 padding을 추가하는 연산입니다.<br>
````python
paddings[D, 0] + tensor.dim_size(D) + paddings[D, 1]
````
의 규칙을 따릅니다. 각 차원 별로 전후 몇개의 padding을 결정합니다.<br>


### (2) C1 구성하기

텐서플로우에서는 convolution 연산자를 지원해줍니다.<br>
````python
tf.nn.conv2d(input, filter, strides, padding,...)
````
으로 구성되어 있습니다. 이때 input과 filter는 아래의 shape 순서를 지켜야 합니다.

1. input : [batch, in_height, in_width, in_channels]
2. filter : [filter_height, filter_width, in_channels, out_channels]

In [0]:
filter_height = 5#fix me #
filter_width = 5#fix me #
in_channels = 1#fix me #
out_channels = 6#fix me #

with graph.as_default():
    with tf.variable_scope('C1'):
        
        xa_init = tf.initializers.glorot_normal()#fix me#
        w1_init = xa_init((filter_height, filter_width, in_channels, out_channels))#fix me#
        b1_init = tf.zeros((out_channels,))#fix me#
        
        w1 = tf.Variable(w1_init)
        b1 = tf.Variable(b1_init)
        
        c1 = tf.nn.conv2d(x_with_pad, w1, [1,1,1,1], 'VALID')
        c1 = c1 + b1#fix me#
        c1 = tf.nn.tanh(c1)#fix me# : activation tanh

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
print("출력의 형태 :",c1.shape)

출력의 형태 : (?, 28, 28, 6)


In [0]:
show_graph(graph)

### (3) S2 구성하기

텐서플로우에서는 pooling 연산자를 지원해줍니다.<br>
논문에서는 average Pooling으로 Max Pooling과 달리 평균값을 이용해주는 기법입니다.<br>
Max Pooling이 좀 더 성능이 좋으므로 변경하겠습니다.
````python
tf.nn.max_pool(value, ksize, strides, padding,...)
````
으로 구성되어 있습니다. 이때 input과 filter는 아래의 shape 순서를 지켜야 합니다.

1. value : [batch, in_height, in_width, in_channels]
2. ksize : 각 차원 별 ksize -> batch 별, 채널 별로 적용하므로 [1,in_height,in_width,1]이 되어야 함

In [0]:
k = 2 # pooling의 크기
with graph.as_default():
    with tf.variable_scope('S2'):
        s2 = tf.nn.max_pool(c1, [1,k,k,1], [1,k,k,1], 'VALID')

In [0]:
print("출력의 형태 :",s2.shape)

출력의 형태 : (?, 14, 14, 6)


In [0]:
show_graph(graph)

### (4) C3 구성하기

C1과 동일하게 구성하여 줍니다.

In [0]:
filter_height = 5
filter_width = 5
in_channels = 6
out_channels = 16

with graph.as_default():
    with tf.variable_scope('C3'):
        xa_init = tf.initializers.glorot_normal()#fix me#
        w3_init = xa_init((filter_height, filter_width, in_channels, out_channels))#fix me#
        b3_init = tf.zeros((out_channels,))#fix me#
        
        w3 = tf.Variable(w3_init)
        b3 = tf.Variable(b3_init)
        
        c3 = tf.nn.conv2d(s2, w3, [1,1,1,1], 'VALID')
        c3 = c3 + b3#fix me#
        c3 = tf.nn.tanh(c3)#fix me# : activation tanh
       

In [0]:
print("출력의 형태 :",c3.shape)

출력의 형태 : (?, 10, 10, 16)


In [0]:
show_graph(graph)

### (4) S4 구성하기

S2과 동일하게 구성하여 줍니다.

In [0]:
k = 2 # pooling의 크기

with graph.as_default():
    with tf.variable_scope('S4'):
        s4 = tf.nn.max_pool(c3, [1,k,k,1], [1,k,k,1], 'VALID')

In [0]:
print("출력의 형태 :",s4.shape)

출력의 형태 : (?, 5, 5, 16)


In [0]:
show_graph(graph)

### (5) C5 구성하기

C5은 Convolution 연산이지만, 실제로는 Fully Connected Layer와 동일합니다<br>
입력 특징맵의 크기와 Filter의 크기가 동일해, 모두 연결되어 있기 때문입니다.

In [0]:
filter_height = 5#fix me#
filter_width = 5#fix me #
in_channels = 16#fix me#
out_channels = 120#fix me#

with graph.as_default():
    with tf.variable_scope('S5'):
        xa_init = tf.initializers.glorot_normal()#fix me#
        w5_init = xa_init((filter_height, filter_width, in_channels, out_channels))#fix me#
        b5_init = tf.zeros((out_channels,))#fix me#
        
        w5 = tf.Variable(w5_init)
        b5 = tf.Variable(b5_init)
        
        c5 = tf.nn.conv2d(s4, w5, [1,1,1,1], 'VALID')
        c5 = c5 + b5#fix me#
        c5 = tf.nn.tanh(c5)#fix me# : activation tanh

In [0]:
print("출력의 형태 :",c5.shape)

출력의 형태 : (?, 1, 1, 120)


In [0]:
c5

<tf.Tensor 'S5/Tanh:0' shape=(?, 1, 1, 120) dtype=float32>

In [0]:
show_graph(graph)

### (5) F6 구성하기

F6은 이전에 배웠던 Fully Connected Layer입니다.<Br>
우선 Fully Connected Layer을 적용하기 위해서는 Tensor의 형태를 바꾸어 주어야 합니다.

In [0]:
in_channels = 120
out_channels = 84

with graph.as_default():
    with tf.variable_scope('F6'):
#         c5 = tf.layers.Flatten()(c5)#fix me # : flatten c5 
        c5 = tf.reshape(c5, [-1, in_channels])
        f6 = tf.layers.Dense(out_channels)(c5)#fix me # : dense layer , activation tanh
        


In [0]:
print("출력의 형태 :",f6.shape)

출력의 형태 : (?, 84)


In [0]:
show_graph(graph)

### (6) OUTPUT 구성하기

Original Paper에서는 RBF(Euclidean Radial Basis Function)을 이용하였습니다. <br>
현대 딥러닝에서는 대부분의 분류기 모델에서는 주로 softmax와 Cross-Entropy를 이용하여<br>
학습시킵니다.

In [0]:
num_classes = 10 

with graph.as_default():
    with tf.variable_scope('OUTPUT'):
        logits = #fix me# 
    y_pred = #fix me# : softmax 
    

SyntaxError: ignored

In [0]:
print("출력의 형태 :",logits.shape)

In [0]:
show_graph(graph)

### (6) 학습에 필요한 연산자 구현하기 

우리의 출력 활성화함수가 softmax이므로, softmax-cross-entropy를 이용하도록 하겠습니다.<br>
그리고 optimizer도 adam optimizer을 이용하도록 하겠습니다.



In [0]:
with graph.as_default():
    y_true = #fix me#
    lr = #fix me #
    
    loss = # fix me # use tf.losses.sparse_softmax_cross_entropy()
    
    train_op = #fix me #

sparse_softmax_cross_entropy는 y_true 값으로 one-hot vector을 넣는 것이 아닌<br>
class 라벨(ex) 0, 3, 2..)을 넣습니다.

In [0]:
show_graph(graph)

### (7) Metric 구현하기

분류기의 성능을 평가하기 위해, 정확도에 관련된 연산자를 구현하겠습니다.

In [0]:
with graph.as_default():
    with tf.variable_scope('accuracy'):
        pred = # fix me #
        correct = # fix me #
        accuracy = # fix me #

In [0]:
show_graph(graph)

<br>

## 2. LeNet 학습시키기
----

mnist을 불러와 모델을 학습시키도록 하겠습니다.

### (1) 데이터 가져오기

In [0]:
from tensorflow.keras.datasets.mnist import load_data

(train_X, train_Y), (test_X, test_Y) = load_data()
train_X = train_X/255. # 0~1 정규화
train_X = np.expand_dims(train_X,axis=-1)
test_X = test_X/255. # 0~1 정규화
test_X = np.expand_dims(test_X,axis=-1)

In [0]:
idx = 15
plt.title(train_Y[idx])
plt.imshow(train_X[idx,:,:,0])
plt.show()


### (2) 모델 학습시키기

In [0]:
num_epoch = 10
num_batch = 100
num_step = len(train_X)//num_batch

with graph.as_default():

    sess = tf.Session(graph=graph)
    sess.run(tf.global_variables_initializer())

    for epoch in range(num_epoch):
        for step in tqdm(range(num_step)):
            batch_X = train_X[step*num_batch:(step+1)*num_batch]
            batch_Y = train_Y[step*num_batch:(step+1)*num_batch]
            sess.run(train_op,feed_dict={
                x:batch_X,
                y_true:batch_Y
            })

        acc = sess.run(accuracy,feed_dict={
            x:test_X,
            y_true:test_Y
        })
        print("[{:2d} epoch] Test 정확도 : {:2.2f}%"
              .format(epoch+1, acc*100))
